## Import

In [1]:
import sys
sys.path.append(r'C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\src\facility_location_Bergen\custome_modules')

In [2]:
import warnings
from shapely.errors import ShapelyDeprecationWarning
# Ignore the ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [3]:
import io
import copy
import pytz
import folium
import pyproj
import numpy as np
import pandas as pd
import pickle as pkl
from PIL import Image
import networkx as nx
from time import time
import geopandas as gpd
import cartopy.crs as ccrs
from datetime import datetime
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import cartopy.io.img_tiles as cimgt
from shapely.geometry import MultiPoint
from urllib.request import urlopen, Request
from matplotlib.animation import FuncAnimation
from mongo_db import retrieve_database_and_collections
from networkx.drawing.nx_agraph import graphviz_layout
from convert_geometry import toMultiLineString, toExtremePoints
from log import print_INFO_message_timestamp, print_INFO_message
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString
from facility_location import AdjacencyMatrix, FacilityLocation, FacilityLocationReport

## Retrieve collection and convert to a dataframe

In [4]:
day = "20_04_2023"

In [5]:
def get_gdf(day):
    with open(rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\{day}.geojson", "rb") as f:
        gdf = pkl.load(f)
    return gdf

In [6]:
gdf = get_gdf(day)

## Data Exploration 

In [7]:
gdf.columns

Index(['sourceUpdated', 'api_call_time', 'geometry_length', 'description',
       'length', 'currentFlow.speed', 'currentFlow.speedUncapped',
       'currentFlow.freeFlow', 'currentFlow.jamFactor',
       'currentFlow.confidence', 'currentFlow.traversability', 'geometry.type',
       'currentFlow.subSegments', 'currentFlow.jamTendency',
       'currentFlow.junctionTraversability', 'geometry.multi_line',
       'geometry.extreme_point'],
      dtype='object')

In [29]:
list(gdf.api_call_time.unique())[0]

Timestamp('2023-04-20 13:00:00+0000', tz='UTC')

In [30]:
gdf_ = gdf[gdf.api_call_time == list(gdf.api_call_time.unique())[0]]

In [33]:
print(len(gdf_), len(gdf_["geometry.extreme_point"].unique()))

620 616


### Subsegments

In [8]:
gdf.head()

,sourceUpdated,api_call_time,geometry_length,description,length,currentFlow.speed,currentFlow.speedUncapped,currentFlow.freeFlow,currentFlow.jamFactor,currentFlow.confidence,currentFlow.traversability,geometry.type,currentFlow.subSegments,currentFlow.jamTendency,currentFlow.junctionTraversability,geometry.multi_line,geometry.extreme_point
_id,,,,,,,,,,,,,,,,,
6453c60a68f2b2bf1c94ab37,2023-04-20 12:58:09+00:00,2023-04-20 13:00:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.611112,13.611112,16.11111,1.4,0.74,open,GeometryCollection,NaN,NaN,NaN,"MULTILINESTRING ((5.53747 60.42335, 5.53757 60...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
6453c60a68f2b2bf1c94af46,2023-04-20 13:03:09+00:00,2023-04-20 13:05:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.055556,13.055556,16.11111,1.7,0.72,open,GeometryCollection,NaN,NaN,NaN,"MULTILINESTRING ((5.53747 60.42335, 5.53757 60...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
6453c60a68f2b2bf1c94b12a,2023-04-20 13:08:09+00:00,2023-04-20 13:10:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.055556,13.055556,16.11111,1.6,0.72,open,GeometryCollection,NaN,NaN,NaN,"MULTILINESTRING ((5.53747 60.42335, 5.53757 60...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
6453c60a68f2b2bf1c94b64c,2023-04-20 13:13:09+00:00,2023-04-20 13:15:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.055556,13.055556,16.11111,1.6,0.71,open,GeometryCollection,NaN,NaN,NaN,"MULTILINESTRING ((5.53747 60.42335, 5.53757 60...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
6453c60a68f2b2bf1c94b796,2023-04-20 13:18:09+00:00,2023-04-20 13:20:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.055556,13.055556,16.11111,1.6,0.71,open,GeometryCollection,NaN,NaN,NaN,"MULTILINESTRING ((5.53747 60.42335, 5.53757 60...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."


In [9]:
print_INFO_message_timestamp("Number of data: {}".format(len(gdf)))
print_INFO_message("Number of data with no subsegments current flow: {}".format(gdf["currentFlow.subSegments"].isna().sum()))
print_INFO_message("Number of data with subsegments current flow: {}".format(len(gdf) - gdf["currentFlow.subSegments"].isna().sum()))

[05/09/23 14:42:09] INFO     Number of data: 39107
                    INFO     Number of data with no subsegments current flow: 30083
                    INFO     Number of data with subsegments current flow: 9024


In [10]:
geometry_with_subsegment = gdf.where(gdf["currentFlow.subSegments"].isna()==False).dropna(how="all")["geometry.multi_line"].unique()

In [11]:
print_INFO_message_timestamp("Number of geometries with subsegments current flow: {}".format(len(geometry_with_subsegment)))
for geo in geometry_with_subsegment:
    sub_gdf = gdf[gdf["geometry.multi_line"]==geo]
    sub_gdf_not_null = sub_gdf.where(sub_gdf["currentFlow.subSegments"].isna()==False).dropna(how="all")
    print_INFO_message("Number of dates with subsegments current flow: {} out of {}".format(len(sub_gdf_not_null), len(sub_gdf)))

[05/09/23 14:42:12] INFO     Number of geometries with subsegments current flow: 336
                    INFO     Number of dates with subsegments current flow: 2 out of 63
                    INFO     Number of dates with subsegments current flow: 63 out of 63
                    INFO     Number of dates with subsegments current flow: 63 out of 63
                    INFO     Number of dates with subsegments current flow: 7 out of 63
                    INFO     Number of dates with subsegments current flow: 4 out of 63
                    INFO     Number of dates with subsegments current flow: 63 out of 63
                    INFO     Number of dates with subsegments current flow: 3 out of 63
                    INFO     Number of dates with subsegments current flow: 1 out of 63
                    INFO     Number of dates with subsegments current flow: 1 out of 63
                    INFO     Number of dates with subsegments current flow: 1 out of 63
                    INFO     Num

### Traversability

In [16]:
gdf["currentFlow.traversability"].value_counts()

currentFlow.traversability
open      38882
closed      225
Name: count, dtype: int64

In [17]:
gdf["currentFlow.junctionTraversability"].value_counts()

currentFlow.junctionTraversability
intermediateClosedEdgeOpen    225
Name: count, dtype: int64

In [18]:
bol = (gdf[gdf["currentFlow.traversability"]=="closed"].index == gdf[gdf["currentFlow.junctionTraversability"]=="intermediateClosedEdgeOpen"].index).all()
print_INFO_message_timestamp("Are the closed traversability and the intermediateClosedEdgeOpen traversability the same?")
print_INFO_message(bol)

[05/09/23 14:26:52] INFO     Are the closed traversability and the intermediateClosedEdgeOpen traversability the same?
                    INFO     True


In [19]:
closed_road = gdf[gdf["currentFlow.traversability"]=="closed"]["geometry.multi_line"].unique()
print_INFO_message_timestamp("Number of closed roads:")
print_INFO_message(len(closed_road))

[05/09/23 14:26:52] INFO     Number of closed roads:
                    INFO     9


## Data preparation

### Find unique identifier for road segments

In [35]:
for date in gdf["api_call_time"].unique():
    sub_gdf = gdf[gdf["api_call_time"] == date]
    key1 = sub_gdf["geometry.multi_line"]
    key2 = sub_gdf["geometry.extreme_point"]
    print(date, len(sub_gdf)-len(key1.unique()), len(key1.unique())-len(key2.unique()))

2023-04-20 13:00:00+00:00 4 0
2023-04-20 13:05:00+00:00 4 0
2023-04-20 13:10:00+00:00 4 0
2023-04-20 13:15:00+00:00 4 0
2023-04-20 13:20:00+00:00 4 0
2023-04-20 13:25:00+00:00 4 0
2023-04-20 13:30:00+00:00 4 0
2023-04-20 13:35:00+00:00 4 0
2023-04-20 13:40:00+00:00 4 0
2023-04-20 13:45:00+00:00 4 0
2023-04-20 13:50:00+00:00 4 0
2023-04-20 13:55:00+00:00 4 0
2023-04-20 14:00:00+00:00 4 0
2023-04-20 14:05:00+00:00 4 0
2023-04-20 14:10:00+00:00 4 0
2023-04-20 14:15:00+00:00 4 0
2023-04-20 14:20:00+00:00 4 0
2023-04-20 14:25:00+00:00 4 0
2023-04-20 14:30:00+00:00 4 0
2023-04-20 14:35:00+00:00 4 0
2023-04-20 14:40:00+00:00 4 0
2023-04-20 14:45:00+00:00 4 0
2023-04-20 14:50:00+00:00 4 0
2023-04-20 14:55:00+00:00 4 0
2023-04-20 15:00:00+00:00 4 0
2023-04-20 09:00:00+00:00 4 0
2023-04-20 09:05:00+00:00 4 0
2023-04-20 09:10:00+00:00 4 0
2023-04-20 09:15:00+00:00 4 0
2023-04-20 09:20:00+00:00 4 0
2023-04-20 09:25:00+00:00 4 0
2023-04-20 09:30:00+00:00 4 0
2023-04-20 09:35:00+00:00 4 0
2023-04-20

In [10]:
gdf = gdf.set_geometry("geometry.extreme_point")

In [11]:
for date in list(gdf["api_call_time"].unique()):
    sub_gdf = gdf[(gdf["api_call_time"] == date)]
    vc = sub_gdf["geometry.extreme_point"].value_counts(sort=False)
    sub_sub_gdf = sub_gdf[sub_gdf["geometry.extreme_point"].isin(vc[vc > 1].index)]
    for geometry in sub_sub_gdf["geometry.extreme_point"].unique():
        print("the description field is different: ", len(sub_sub_gdf[sub_sub_gdf["geometry.extreme_point"] == geometry]["description"].unique())>1)
        if len(sub_sub_gdf[sub_sub_gdf["geometry.extreme_point"] == geometry]["description"].unique())<2:
            print(date)
            break

the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different:  True
the description field is different

From the previous cells we observe that the tuple ("description, "geometry") is unique for each road segment. We can therefore use this tuple as a unique identifier for each road segment. 

### Deal with subsegments

In [12]:
print_INFO_message_timestamp("Start")
for i in range(len(gdf)):
    if i%10000 == 0:
        print_INFO_message("Index: {}".format(i))
    if not sum(gdf.iloc[i]["geometry_length"]) == gdf.iloc[i]["length"]:
        print_INFO_message("False at index: {}".format(i))
print_INFO_message_timestamp("End")

[07/10/23 11:28:27] INFO     Start
                    INFO     Index: 0
                    INFO     Index: 10000
                    INFO     Index: 20000
                    INFO     Index: 30000
[07/10/23 11:29:04] INFO     End


In [15]:
gdf_with_subsegments = gdf.where(gdf["currentFlow.subSegments"].isna()==False).dropna(how="all")

In [52]:
def get_length(data):
    n = len(data["currentFlow.subSegments"])
    length_list = []
    index = 0

    # print_INFO_message_timestamp("Start")

    for i in range(n):
        # print_INFO_message("Index: {}".format(index))
        length_list.append([])
        length = data["currentFlow.subSegments"][i]["length"]
        
        for j in range(index, len(data["geometry_length"])):
            # print(j, len(data["geometry_length"]))
            length_list[i].append(data["geometry_length"][j])
            rel_error = abs(length - sum(length_list[i]))/length
            
            if j < len(data["geometry_length"])-2:
                next = sum(length_list[i]) + data["geometry_length"][j+1]
                rel_error_next = abs(length - next)/length
                
                if rel_error_next > rel_error:
                    index = j+1
                    break
    
    # if the total length of the subsegments is not equal to the length of the road
    # we avoid to split the road in subsegments
    # the previous happens in two situatons:
    # 1) the approximation of the final segment get worse when adding the last length
    # 2) the approximation of an intermediate segments improves always when adding the next length
    
    if sum([len(x) for x in length_list]) != len(data["geometry_length"]):
        # print([len(x) for x in length_list])
        # print([len(length_list), n])
        # print([len(data["geometry_length"]), len(data["geometry.multi_line"])])
        return None
    
    total_difference = 0
    for i in range(n):
        total_difference += abs(data["currentFlow.subSegments"][i]["length"] - sum(length_list[i]))
        relative_difference = total_difference/data["length"]
    if relative_difference > 0.01:
        print_INFO_message("Relative difference: {}".format(relative_difference))
        return None
        
    return length_list

In [39]:
def get_geometry(data, length_list):
    n = len(data["currentFlow.subSegments"])
    geometry_list = []
    
    # print_INFO_message_timestamp("Start")
    
    counter = 0
    
    for i in range(n):
        # print_INFO_message("Index: {}".format(i))
        geometry_list.append([])
        for j in range(len(length_list[i])):
            geometry_list[i].append(data["geometry.multi_line"][counter])
            counter += 1
            
        geometry_list[i] = geometry_list[i]
    return geometry_list

In [45]:
def new_rows(data):
    n = len(data["currentFlow.subSegments"])
    new_rows = [{k: np.nan for k in gdf.columns} for i in range(n)]
    length_list = get_length(data)
        
    if length_list is None:
        return None
        
    geometry_list = get_geometry(data, length_list)
        
    for i in range(n):
        new_rows[i]["geometry.multi_line"] = [geometry_list[i]]
        new_rows[i]["geometry.extreme_point"] = [toExtremePoints(MultiLineString(geometry_list[i]))]
        new_rows[i]["geometry_length"] = [length_list[i]]
        new_rows[i]["currentFlow.subSegments"] = np.nan
        new_rows[i]["currentFlow.traversability"] = data["currentFlow.subSegments"][i]["traversability"]
            
        try:
            new_rows[i]["currentFlow.junctionTraversability"] = data["currentFlow.subSegments"][i]["junctionTraversability"]
        except:
            new_rows[i]["currentFlow.junctionTraversability"] = np.nan
                
        try:
            new_rows[i]["currentFlow.jamTendency"] = data["currentFlow.subSegments"][i]["jamTendency"]
        except:
            new_rows[i]["currentFlow.jamTendency"] = np.nan
            
        try:
            new_rows[i]["currentFlow.confidence"] = data["currentFlow.subSegments"][i]["confidence"]
        except:
            new_rows[i]["currentFlow.confidence"] = np.nan
        
        try:
            new_rows[i]["currentFlow.speed"] = data["currentFlow.subSegments"][i]["speed"]
        except:
            new_rows[i]["currentFlow.speed"] = np.nan
        
        try: 
            new_rows[i]["currentFlow.speedUncapped"] = data["currentFlow.subSegments"][i]["speedUncapped"]
        except:
            new_rows[i]["currentFlow.speedUncapped"] = np.nan
            
        new_rows[i]["length"] = data["currentFlow.subSegments"][i]["length"]
        new_rows[i]["currentFlow.freeFlow"] = data["currentFlow.subSegments"][i]["freeFlow"]
        new_rows[i]["currentFlow.jamFactor"] = data["currentFlow.subSegments"][i]["jamFactor"]
        
        new_rows[i]["api_call_time"] = data["api_call_time"]
        new_rows[i]["sourceUpdated"] = data["sourceUpdated"]
        new_rows[i]["description"] = data["description"]     
        new_rows[i]["geometry.type"] = data["geometry.type"]
        
    df_list = []
        
    for i in range(n):
        df_list.append(pd.DataFrame(new_rows[i]))
        df_list[-1]["geometry.multi_line"] = df_list[-1]["geometry.multi_line"].apply(lambda x: MultiLineString(x))
    
    df = pd.concat(df_list, ignore_index=True)
    
    if len(df) != n:
        print("What the fuck")
    return df

In [41]:
def sub_segment_extraction(gdf):
    new_rows_ = []
    indexes = []

    print_INFO_message_timestamp("Start")

    for i in range(len(gdf)):
        data = gdf.iloc[i]
        try:
            isna = np.isnan(data["currentFlow.subSegments"])
        except:
            isna = False
        
        if isna == False:
            print_INFO_message("Index: {}".format(i))
            nr = new_rows(data)
            if nr is not None:
                new_rows_.append(nr)
                indexes.append(i)

    gdf_processed = gdf.reset_index(drop=True).drop(index = indexes)
    gdf_processed = pd.concat([gdf_processed] + new_rows_, ignore_index=True).reset_index(drop=True)
    
    if gdf_processed["currentFlow.subSegments"].isna().all():
        gdf_processed = gdf_processed[['sourceUpdated',
                                        'api_call_time',
                                        'geometry_length',
                                        'description',
                                        'length',
                                        'currentFlow.speed',
                                        'currentFlow.speedUncapped',
                                        'currentFlow.freeFlow',
                                        'currentFlow.jamFactor',
                                        'currentFlow.confidence',
                                        'currentFlow.traversability',
                                        'geometry.type',
                                        'currentFlow.jamTendency',
                                        'currentFlow.junctionTraversability',
                                        'geometry.multi_line',
                                        'geometry.extreme_point']]
    
    return gdf_processed

In [42]:
def process_closed_roads(gdf):
    gdf_copy = copy.deepcopy(gdf)
    gdf_ = gdf_copy[gdf_copy["currentFlow.traversability"]=="closed"]
    indexes = gdf_.index
    for idx in indexes:
        gdf_copy.loc[idx, "currentFlow.speed"] = 0
        gdf_copy.loc[idx, "currentFlow.speedUncapped"] = 0
    
    return gdf_copy

In [43]:
def process_gdf(gdf):
    gdf_ = sub_segment_extraction(gdf)
    gdf_ = process_closed_roads(gdf_)
    return gdf_

In [53]:
gdf_processed = process_gdf(gdf)

[05/09/23 15:11:42] INFO     Start
                    INFO     Index: 59
                    INFO     Index: 60
                    INFO     Index: 190
                    INFO     Relative difference: 0.028687515390297957
                    INFO     Index: 191
                    INFO     Index: 194
                    INFO     Index: 196
                    INFO     Index: 197
                    INFO     Index: 200
                    INFO     Index: 201
                    INFO     Index: 203
                    INFO     Index: 206
                    INFO     Index: 207
                    INFO     Index: 209
                    INFO     Index: 212
                    INFO     Index: 213
                    INFO     Index: 216
                    INFO     Index: 217
                    INFO     Index: 218
                    INFO     Index: 221
                    INFO     Index: 222
                    INFO     Index: 225
                    INFO     Index: 227
                

In [54]:
gdf_processed

,sourceUpdated,api_call_time,geometry_length,description,length,currentFlow.speed,currentFlow.speedUncapped,currentFlow.freeFlow,currentFlow.jamFactor,currentFlow.confidence,currentFlow.traversability,geometry.type,currentFlow.subSegments,currentFlow.jamTendency,currentFlow.junctionTraversability,geometry.multi_line,geometry.extreme_point
0,2023-04-20 12:58:09+00:00,2023-04-20 13:00:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.611112,13.611112,16.111110,1.4,0.74,open,GeometryCollection,NaN,NaN,NaN,"(LINESTRING (5.53747 60.42335, 5.53757 60.4231...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
1,2023-04-20 13:03:09+00:00,2023-04-20 13:05:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.055556,13.055556,16.111110,1.7,0.72,open,GeometryCollection,NaN,NaN,NaN,"(LINESTRING (5.53747 60.42335, 5.53757 60.4231...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
2,2023-04-20 13:08:09+00:00,2023-04-20 13:10:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.055556,13.055556,16.111110,1.6,0.72,open,GeometryCollection,NaN,NaN,NaN,"(LINESTRING (5.53747 60.42335, 5.53757 60.4231...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
3,2023-04-20 13:13:09+00:00,2023-04-20 13:15:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.055556,13.055556,16.111110,1.6,0.71,open,GeometryCollection,NaN,NaN,NaN,"(LINESTRING (5.53747 60.42335, 5.53757 60.4231...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
4,2023-04-20 13:18:09+00:00,2023-04-20 13:20:00+00:00,"[21.0, 60.0, 26.0, 93.0, 14.0, 17.0, 79.0, 64.0]",Herland,374.0,13.055556,13.055556,16.111110,1.6,0.71,open,GeometryCollection,NaN,NaN,NaN,"(LINESTRING (5.53747 60.42335, 5.53757 60.4231...","(POINT (5.53747 60.42335), POINT (5.53903 60.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51851,2023-04-20 07:23:09+00:00,2023-04-20 07:25:00+00:00,"[53.0, 100.0, 219.0, 166.0, 24.0]",Ranetunnelen,561.0,10.833334,13.333334,16.388890,1.7,0.77,open,GeometryCollection,NaN,NaN,NaN,"(LINESTRING (5.2643 60.50011, 5.26428 60.49996...","(POINT (5.2643 60.50011), POINT (5.26306 60.49..."
51852,2023-04-20 07:28:09+00:00,2023-04-20 07:30:00+00:00,"[11.0, 55.0, 23.0, 45.0, 24.0, 14.0, 28.0, 15....",Ranetunnelen,700.0,8.333334,8.333334,8.055555,0.0,0.75,open,GeometryCollection,NaN,NaN,NaN,"(LINESTRING (5.26737 60.5059, 5.26732 60.5058)...","(POINT (5.26737 60.5059), POINT (5.2643 60.500..."
51853,2023-04-20 07:28:09+00:00,2023-04-20 07:30:00+00:00,"[53.0, 100.0, 219.0, 166.0, 24.0]",Ranetunnelen,561.0,10.833334,13.333334,16.388890,1.6,0.75,open,GeometryCollection,NaN,NaN,NaN,"(LINESTRING (5.2643 60.50011, 5.26428 60.49996...","(POINT (5.2643 60.50011), POINT (5.26306 60.49..."
51854,2023-04-20 07:28:09+00:00,2023-04-20 07:30:00+00:00,"[219.0, 100.0, 53.0]",Salhus,371.0,8.055555,8.055555,15.833334,4.9,0.80,open,GeometryCollection,NaN,NaN,NaN,"(LINESTRING (5.26324 60.49688, 5.26308 60.4973...","(POINT (5.26324 60.49688), POINT (5.2643 60.50..."


In [55]:
print_INFO_message_timestamp("Start")
for i in range(len(gdf_processed)):
    if i%10000 == 0:
        print_INFO_message("Index: {}".format(i))
    if not sum(gdf_processed.iloc[i]["geometry_length"]) == gdf_processed.iloc[i]["length"]:
        print_INFO_message("False at index: {}".format(i))
        print_INFO_message("Relatice difference: {}".format((sum(gdf_processed.iloc[i]["geometry_length"]) - gdf_processed.iloc[i]["length"])/gdf_processed.iloc[i]["length"]))
print_INFO_message_timestamp("End")

[05/09/23 15:14:06] INFO     Start
                    INFO     Index: 0
                    INFO     Index: 10000
                    INFO     Index: 20000
                    INFO     Index: 30000
                    INFO     False at index: 31487
                    INFO     Relatice difference: -0.004975124378109453
                    INFO     False at index: 31488
                    INFO     Relatice difference: 0.005780346820809248
                    INFO     False at index: 31489
                    INFO     Relatice difference: -0.004975124378109453
                    INFO     False at index: 31490
                    INFO     Relatice difference: 0.005780346820809248
                    INFO     False at index: 31491
                    INFO     Relatice difference: 0.0013192612137203166
                    INFO     False at index: 31492
                    INFO     Relatice difference: -0.002150537634408602
                    INFO     False at index: 31493
              